In [1]:
from mido import MidiFile

In [3]:
mid = MidiFile('./Dataset-RAW/Beethoven_Piano_Concerto_No_4,_3rd_Mvmt_(Arr_for_2_pianos).mid')
print(len(mid.tracks))

4


In [7]:
for i, track in enumerate(mid.tracks):
    print('Track {}: {}'.format(i, track.name))
    for message in track:
        print(message)

Track 0: 
MetaMessage('time_signature', numerator=2, denominator=4, clocks_per_click=24, notated_32nd_notes_per_beat=8, time=0)
MetaMessage('key_signature', key='G', time=0)
MetaMessage('set_tempo', tempo=416667, time=0)
control_change channel=0 control=121 value=0 time=0
program_change channel=0 program=0 time=0
control_change channel=0 control=7 value=100 time=0
control_change channel=0 control=10 value=64 time=0
control_change channel=0 control=91 value=0 time=0
control_change channel=0 control=93 value=0 time=0
MetaMessage('midi_port', port=0, time=0)
note_on channel=0 note=76 velocity=64 time=9600
note_on channel=0 note=76 velocity=0 time=455
note_on channel=0 note=76 velocity=64 time=145
note_on channel=0 note=76 velocity=0 time=113
note_on channel=0 note=79 velocity=64 time=7
note_on channel=0 note=79 velocity=0 time=113
note_on channel=0 note=77 velocity=64 time=7
note_on channel=0 note=77 velocity=0 time=113
note_on channel=0 note=76 velocity=64 time=7
note_on channel=0 note=7

In [6]:
cont = 0
for m in mid.tracks:
    for mex in m:
        if(not 'control_change' in str(mex)):
            cont += 1
            if(cont > 15):
                break
            print(mex)
    print("----------------------")
    cont = 0


MetaMessage('time_signature', numerator=2, denominator=4, clocks_per_click=24, notated_32nd_notes_per_beat=8, time=0)
MetaMessage('key_signature', key='G', time=0)
MetaMessage('set_tempo', tempo=416667, time=0)
program_change channel=0 program=0 time=0
MetaMessage('midi_port', port=0, time=0)
note_on channel=0 note=76 velocity=64 time=9600
note_on channel=0 note=76 velocity=0 time=455
note_on channel=0 note=76 velocity=64 time=145
note_on channel=0 note=76 velocity=0 time=113
note_on channel=0 note=79 velocity=64 time=7
note_on channel=0 note=79 velocity=0 time=113
note_on channel=0 note=77 velocity=64 time=7
note_on channel=0 note=77 velocity=0 time=113
note_on channel=0 note=76 velocity=64 time=7
note_on channel=0 note=76 velocity=0 time=455
----------------------
MetaMessage('key_signature', key='G', time=0)
MetaMessage('midi_port', port=0, time=0)
note_on channel=0 note=48 velocity=40 time=9600
note_on channel=0 note=48 velocity=0 time=227
note_on channel=0 note=55 velocity=48 time

In [70]:
streams = list()
for m in mid.tracks:
    streams.append(m)

In [71]:
def get_notes_from_midi(file_path, channel):
    """
    Extract all notes played by a specific instrument (channel) from a MIDI file.

    :param file_path: Path to the MIDI file
    :param channel: MIDI channel number (0-15) corresponding to the instrument
    :return: List of tuples, each containing (note, velocity, time)
    """
    midi_file = MidiFile(file_path)
    streams = []
    notes = []
    
    for track in midi_file.tracks:
        for msg in track:
            if msg.type == 'note_on' and msg.channel == channel:
                note = msg.note
                velocity = msg.velocity
                time = msg.time
                notes.append((note, velocity, time))
        streams.append(notes)
                
    return notes

In [72]:
def get_notes_from_streams(source):
    """
    Extract all notes played by a specific instrument (channel) from a MIDI file.

    :param source: the MIDI file already extracted and divided in relative streams (one for each instrument)
    :param channel: MIDI channel number (0-15) corresponding to the instrument
    :return: List of tuples, each containing (note, velocity, time)
    """
    streams = []
    notes = []
    
    for track in source:
        for msg in track:
            if msg.type == 'note_on':
                note = msg.note
                velocity = msg.velocity
                time = msg.time
                notes.append((note, velocity, time))
        streams.append(notes)
                
    return streams

In [73]:
def get_notes_with_time(file_path, channels):
    """
    Extract notes and their timings from specified channels.

    :param file_path: Path to the MIDI file
    :param channels: List of two channels (ints) corresponding to two different instruments
    :return: Dictionary with channel as key and list of (note, velocity, time_stamp) as values
    """
    midi_file = mido.MidiFile(file_path)
    notes_data = {channel: [] for channel in channels}
    current_time = {channel: 0 for channel in channels}

    for track in midi_file.tracks:
        for msg in track:
            if msg.type == 'note_on' and msg.channel in channels:
                current_time[msg.channel] += msg.time
                if msg.velocity > 0:  # note on
                    notes_data[msg.channel].append((msg.note, msg.velocity, current_time[msg.channel]))

    return notes_data


In [74]:
def get_notes_with_time_from_stream(sources):
    """
    Extract notes and their timings from specified channels.

    :param file_path: Path to the MIDI file
    :param channels: List of two channels (ints) corresponding to two different instruments
    :return: Dictionary with channel as key and list of (note, velocity, time_stamp) as values
    """
    streams = []
    notes = []

    current_time = 0

    for track in sources:
        for msg in track:
            if msg.type == 'note_on':
                #print("note detected")
                current_time += msg.time
                if msg.velocity > 0:  # note on
                    #print("note played")
                    notes.append((msg.note, msg.velocity, current_time))
        streams.append(notes)
        current_time = 0

    return streams


In [75]:
data = get_notes_from_streams(streams)

In [76]:
data_coll = get_notes_with_time_from_stream(streams)

In [77]:
print(data_coll)

[[(60, 80, 600), (64, 80, 612), (65, 80, 624), (67, 80, 636), (55, 80, 756), (48, 80, 756), (52, 80, 768), (53, 80, 774), (55, 80, 780), (53, 80, 780), (60, 80, 792), (52, 80, 792), (60, 80, 804), (64, 80, 816), (60, 80, 822), (65, 80, 828), (67, 80, 840), (48, 80, 960), (55, 80, 960), (52, 80, 972), (53, 80, 978), (55, 80, 984), (53, 80, 984), (60, 80, 996), (52, 80, 996), (60, 80, 1056), (60, 80, 1068), (64, 80, 1074), (60, 80, 1080), (65, 80, 1086), (67, 80, 1272), (76, 80, 1272), (72, 80, 1278), (74, 80, 1284), (64, 80, 1284), (72, 80, 1290), (60, 80, 1296), (72, 80, 1296), (72, 80, 1302), (67, 80, 1308), (64, 80, 1326), (72, 80, 1338), (60, 80, 1344), (55, 80, 1350), (76, 80, 1350), (64, 80, 1356), (62, 80, 1362), (74, 80, 1482), (74, 80, 1494), (69, 80, 1506), (71, 80, 1554), (64, 80, 1674), (64, 80, 1686), (62, 80, 1698), (60, 80, 1710), (36, 80, 1830), (40, 80, 1842), (43, 80, 1854), (60, 80, 1854), (48, 80, 1866), (64, 80, 1866), (60, 80, 1878), (67, 80, 1884), (64, 80, 1890),

In [78]:
def create_data_windows(notes_data, window_duration = 480):
    """
    Create training data windows and corresponding targets.

    :param notes_data: Dictionary with channels as keys and notes data as values
    :param input_channel: Channel number for input data
    :param target_channel: Channel number for target data
    :param window_duration: Duration of each window in MIDI ticks
    :return: Input sequences and their corresponding target notes
    """
    input_sequences = []
    targets = []

    input_notes = notes_data[1]
    target_notes = notes_data[2]

   # Create input sequences
    start_time = 0
    while start_time < input_notes[-1][2]:  # last note time
        window_end = start_time + window_duration
        window = [note for note in input_notes if start_time <= note[2] < window_end]
        
        # Find the immediate next note in the target after the current window ends
        next_notes = [note for note in target_notes if note[2] >= window_end]
        if next_notes:
            input_sequences.append(window)
            targets.append(next_notes[0])  # the first note after the window
        # If no notes are found in B after the window, the window is discarded

        start_time += window_duration  # non-overlapping windows

    return input_sequences, targets

In [ ]:
def create_data_windowsV2(notes_data, window_size):
    """
    Create training data windows and corresponding targets.

    :param notes_data: Dictionary with channels as keys and notes data as values
    :param input_channel: Channel number for input data
    :param target_channel: Channel number for target data
    :param window_size: Number of notes in each input sequence
    :return: Input sequences and their corresponding target notes
    """
    input_sequences = []
    targets = []

    input_notes = []
    target_notes = []

    for i in range(len(input_notes) - window_size):
        # Get the current window of notes
        window = input_notes[i:i + window_size]
        input_sequences.append(window)
        
        # Find the closest note in target_notes that comes after the last note in the window
        last_note_time = window[-1][2]
        next_note = min(
            (note for note in target_notes if note[2] > last_note_time), 
            key=lambda note: note[2] - last_note_time,
            default=None
        )
        
        if next_note:
            targets.append(next_note)
        else:
            # If no subsequent note is found, drop the window
            input_sequences.pop()

    return input_sequences, targets

# Example usage
midi_path = 'path_to_your_midi_file.mid'
channels = [0, 1]  # Channels for two instruments
window_size = 10  # Number of notes in each input sequence

notes_data = get_notes_with_time_from_stream(streams)
input_sequences, targets = create_data_windows(notes_data,window_size)

print(f"Number of usable sequences: {len(input_sequences)}")
print("Sample input sequence and target:", input_sequences[0], targets[0] if input_sequences else "No usable data")

In [79]:
input_sequences, targets = create_data_windows(data_coll)


In [80]:
import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn

from torch.nn.utils.rnn import pad_sequence

class MIDIDataset(Dataset):
    def __init__(self, input_sequences, targets):
        # Convert each sequence to a tensor when initializing the dataset
        self.inputs = [torch.tensor(sequence, dtype=torch.float32) for sequence in input_sequences]
        self.targets = [torch.tensor(target[0], dtype=torch.long) for target in targets]  # Assuming target is a single note index

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        return self.inputs[idx], self.targets[idx]
    

def collate_fn(batch):
    # Separate sequences and targets from the batch
    sequences, targets = zip(*batch)
    # Pad sequences dynamically
    sequences_padded = pad_sequence([torch.tensor(seq) for seq in sequences], batch_first=True, padding_value=0)
    targets = torch.tensor(targets)
    return sequences_padded, targets


# Example usage
dataset = MIDIDataset(input_sequences, targets)
# Assuming your dataset is defined properly
data_loader = DataLoader(dataset, batch_size=32, shuffle=True, collate_fn=collate_fn)

In [81]:
for inputs, labels in data_loader:
    print(inputs.shape, labels.shape)
    break  # just to see the first batch

RuntimeError: The size of tensor a (3) must match the size of tensor b (0) at non-singleton dimension 1

In [ ]:
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

class MusicLSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim1, hidden_dim2, output_dim):
        super(MusicLSTM, self).__init__()
        self.lstm1 = nn.LSTM(input_dim, hidden_dim1, batch_first=True, dropout=0.1)
        self.lstm2 = nn.LSTM(hidden_dim1, hidden_dim2, batch_first=True)
        self.dropout = nn.Dropout(0.1)
        self.dense1 = nn.Linear(hidden_dim2, 256)
        self.dense2 = nn.Linear(256, output_dim)
        self.softmax = nn.Softmax(dim=1)
    """
    def forward(self, x):
        lengths = x.ne(0).sum(1)  # Calculate lengths of each sequence in the batch
        packed_input = pack_padded_sequence(x, lengths, batch_first=True, enforce_sorted=False)
        packed_output, _ = self.lstm1(packed_input)
        output, _ = pad_packed_sequence(packed_output, batch_first=True)
        output = self.dropout(output)

        _, (hidden, _) = self.lstm2(output)
        x = hidden[-1]

        x = self.dense1(x)
        x = self.dropout(x)
        x = self.dense2(x)
        x = self.softmax(x)
        return x
    """
    
    def forward(self, x):
        # Calculate lengths while ignoring padding
        lengths = torch.sum(x != 0, dim=2).sum(1)  # Make sure it calculates correctly
        print("Lengths:", lengths)  # Debug: Check lengths
        packed_x = pack_padded_sequence(x, lengths.cpu(), batch_first=True, enforce_sorted=False)
        output, _ = self.lstm1(packed_x)
        output, _ = pad_packed_sequence(output, batch_first=True)

        output = self.dropout(output)
        
        _, (hidden, _) = self.lstm2(output)
        x = hidden[-1]  # Take the last hidden state

        x = self.dense1(x)
        x = self.dropout(x)
        x = self.dense2(x)
        x = self.softmax(x)

        print("Output shape:", x.shape)  # Debug: Check output shape
        return x

# Initialize the model
input_dim = 2  # Depends on your actual input features per time step
hidden_dim1 = 128
hidden_dim2 = 64
output_dim = 88  # Assuming piano range
model = MusicLSTM(input_dim, hidden_dim1, hidden_dim2, output_dim)

In [ ]:
from tqdm import tqdm
import torch.optim as optim
import torch.nn as nn

optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

# Number of epochs
num_epochs = 50

for epoch in range(num_epochs):
    # Setup tqdm for the current epoch
    loop = tqdm(data_loader, leave=True)
    for inputs, labels in loop:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        # Update the progress bar description with the latest loss
        loop.set_description(f"Epoch [{epoch+1}/{num_epochs}]")
        loop.set_postfix(loss=loss.item())

    # Close the tqdm loop at the end of each epoch to ensure it doesn't get stuck
    loop.close()

  0%|          | 0/1 [00:00<?, ?it/s]


RuntimeError: The size of tensor a (3) must match the size of tensor b (0) at non-singleton dimension 1